#Data Loading

In [ ]:
import kagglehub
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import re, string
from string import punctuation

# import nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud,STOPWORDS

lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop_words.update(punctuation)

def load_data():
  path = kagglehub.dataset_download("shivamb/real-or-fake-fake-jobposting-prediction")

  os.listdir(path)

  df = pd.read_csv(os.path.join(path, "fake_job_postings.csv"))

  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
df = load_data()
df.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


#Data Preprocessing/Cleanning

In [ ]:
# Drop the 'job_id' column
df = df.drop(columns=['job_id'])

In [ ]:
# Calling object Columns
df.select_dtypes(include='object').columns

Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function'],
      dtype='object')

In [ ]:
 object_columns_df = df.select_dtypes(include='object')

In [ ]:
# Calling Numerical Columns
df.select_dtypes(include='int').columns

Index(['telecommuting', 'has_company_logo', 'has_questions', 'fraudulent'], dtype='object')

In [ ]:
int_columns_df = df.select_dtypes(include='int')

In [ ]:
#df fake and real job postings
fake = df[df['fraudulent'] == 1]
real = df[df['fraudulent'] == 0]
# Count the occurrences of unique values in the 'fraudulent' column
df['fraudulent'].value_counts()

,count
fraudulent,
0,17014
1,866


In [ ]:
df.shape

(17880, 17)

In [ ]:
#Check missing value
df.isnull().sum()

,0
title,0
location,346
department,11547
salary_range,15012
company_profile,3308
description,1
requirements,2696
benefits,7212
telecommuting,0
has_company_logo,0


In [ ]:
#filling them with space -> ' '
df.fillna(" ", inplace=True)
df.isnull().sum()

,0
title,0
location,0
department,0
salary_range,0
company_profile,0
description,0
requirements,0
benefits,0
telecommuting,0
has_company_logo,0


In [ ]:
# List of columns to concatenate
columns_to_concat = ['title', 'company_profile', 'description', 'requirements', 'benefits']

# Concatenate the values of specified columns into a new column 'job_posting'
df['job_posting'] = df[columns_to_concat].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Create a new DataFrame with columns 'job_posting' and 'fraudulent'
new_df = df[['job_posting', 'fraudulent']].copy()

new_df.head(3)

,job_posting,fraudulent
0,"Marketing Intern We're Food52, and we've creat...",0
1,Customer Service - Cloud Video Production 90 S...,0
2,Commissioning Machinery Assistant (CMA) Valor ...,0


In [ ]:
# text preprocessing(cleaning)
nltk.download('stopwords')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove digits
    text = re.sub(r'\d', '', text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word.lower() not in stop_words]
    text = ' '.join(words)

    return text

# Apply the combined function to the 'job_posting' column
new_df['job_posting'] = new_df['job_posting'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
Fake = new_df[new_df['fraudulent'] == 1]

,job_posting,fraudulent
98,ice technician staffing amp recruiting done ri...,1
144,forward cap group raised fund purchase homes s...,1
173,technician instrument controls edison internat...,1
180,sales executive sales executive sales executiv...,1
215,ice technician mt poso staffing amp recruiting...,1


In [ ]:
Real = new_df[new_df['fraudulent'] == 0]

,job_posting,fraudulent
0,marketing intern food weve created groundbreak...,0
1,customer service cloud video production second...,0
2,commissioning machinery assistant cma valor se...,0
3,account executive washington dc passion improv...,0
4,bill review manager spotsource solutions llc g...,0


#EDA

##Tokeniztion

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
#nltk.download('punkt_tab')
nltk.download('punkt')

# Tokenize each job posting into words
new_df['job_posting_tokens'] = new_df['job_posting'].apply(word_tokenize)

# Tokenize each job posting into sentences
new_df['job_posting_sentences'] = new_df['job_posting'].apply(sent_tokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
Fake = new_df[new_df['fraudulent'] == 1]
Fake.head(3)

,job_posting,fraudulent,job_posting_tokens,job_posting_sentences
98,ice technician staffing amp recruiting done ri...,1,"[ice, technician, staffing, amp, recruiting, d...",[ice technician staffing amp recruiting done r...
144,forward cap group raised fund purchase homes s...,1,"[forward, cap, group, raised, fund, purchase, ...",[forward cap group raised fund purchase homes ...
173,technician instrument controls edison internat...,1,"[technician, instrument, controls, edison, int...",[technician instrument controls edison interna...


In [ ]:
Real = new_df[new_df['fraudulent'] == 0]
Real.head(3)

,job_posting,fraudulent,job_posting_tokens,job_posting_sentences
0,marketing intern food weve created groundbreak...,0,"[marketing, intern, food, weve, created, groun...",[marketing intern food weve created groundbrea...
1,customer service cloud video production second...,0,"[customer, service, cloud, video, production, ...",[customer service cloud video production secon...
2,commissioning machinery assistant cma valor se...,0,"[commissioning, machinery, assistant, cma, val...",[commissioning machinery assistant cma valor s...


##Split data

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new_df['job_posting_tokens'], new_df['fraudulent'], test_size=0.2, random_state=42)

In [ ]:
# for text feature extraction
from sklearn.feature_extraction.text import CountVectorizer

# Flatten the list of lists into a single list of strings
X_train_flattened = [' '.join(sublist) for sublist in X_train]

# Vectorize the text data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_flattened)

# Flatten the list of lists into a single list of strings for test data
X_test_flattened = [' '.join(sublist) for sublist in X_test]
X_test_vec = vectorizer.transform(X_test_flattened)

#Data Modelling

##LR,NB,RF,SVM

In [ ]:
from collections import Counter
print("Original class distribution:", Counter(y_train))

#Additional Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

models = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "SVM": LinearSVC()
}


for name, model in models.items():
    model.fit(X_train_vec, y_train)
    preds = model.predict(X_test_vec)
    print(f"\n{name} Performance:")
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))

Original class distribution: Counter({0: 13619, 1: 685})

Logistic Regression Performance:
Accuracy: 0.9848993288590604
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3395
           1       0.96      0.73      0.83       181

    accuracy                           0.98      3576
   macro avg       0.97      0.86      0.91      3576
weighted avg       0.98      0.98      0.98      3576


Naive Bayes Performance:
Accuracy: 0.9728747203579419
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      3395
           1       0.96      0.49      0.64       181

    accuracy                           0.97      3576
   macro avg       0.96      0.74      0.82      3576
weighted avg       0.97      0.97      0.97      3576


Random Forest Performance:
Accuracy: 0.9815436241610739
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3395
     

##SMOTE(LR+SVM)

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Train-test split
X_train_text, X_test_text, y_train, y_test = train_test_split(
    new_df['job_posting'], new_df['fraudulent'], test_size=0.2, random_state=42
)

# 2. TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = tfidf.fit_transform(X_train_text)
X_test_vec = tfidf.transform(X_test_text)

# 3. Apply SMOTE to training set only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# Check new class balance
print("Original class distribution:", Counter(y_train))
print("Resampled class distribution:", Counter(y_train_resampled))

# 4. Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "SVM": LinearSVC()
}

# 5. Train and evaluate each model on resampled data
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)  # Train on resampled data
    preds = model.predict(X_test_vec)

    print(f"\n{name} Performance:")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Classification Report:\n", classification_report(y_test, preds))

Original class distribution: Counter({0: 13619, 1: 685})
Resampled class distribution: Counter({0: 13619, 1: 13619})

Logistic Regression Performance:
Accuracy: 0.9756711409395973
Confusion Matrix:
 [[3338   57]
 [  30  151]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      3395
           1       0.73      0.83      0.78       181

    accuracy                           0.98      3576
   macro avg       0.86      0.91      0.88      3576
weighted avg       0.98      0.98      0.98      3576


Naive Bayes Performance:
Accuracy: 0.9368008948545862
Confusion Matrix:
 [[3191  204]
 [  22  159]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.94      0.97      3395
           1       0.44      0.88      0.58       181

    accuracy                           0.94      3576
   macro avg       0.72      0.91      0.78      3576
weighted avg       0.97   

### Summary

**Model Performance Comparison (Before vs. After SMOTE)**

| **Model**               | **Accuracy (Before SMOTE)** | **Recall (Class 1) (Before SMOTE)** | **Accuracy (After SMOTE)** | **Recall (Class 1) (After SMOTE)** | **Best For Fraud Detection?** |
|-------------------------|----------------------------|-------------------------------------|----------------------------|------------------------------------|-------------------------------|
| **Logistic Regression** | 98.5%                      | 73%                                 | 97.6%                      | 83%                                | ✅ Better recall after SMOTE   |
| **Naive Bayes**         | 97.3%                      | 49%                                 | 93.7%                      | 88%                                | ❌ Low precision (44%)         |
| **Random Forest**       | 98.1%                      | 62%                                 | 98.1%                      | 64%                                | ❌ Recall didn't improve       |
| **SVM**                 | 98.2%                      | 77%                                 | 98.4%                      | 78%                                | ✅ Best balance overall        |

---

### **Key Insights**
 **SVM**
   - **Before SMOTE:** Strong recall (77%) and accuracy (98.2%).
   - **After SMOTE:** Best balance—**98.4% accuracy, 78% recall, 88% precision**.
   - **Verdict:** **Best overall** for fraud detection.

---

### **Recommendation**
- **For Fraud Detection (High Recall + Precision):**  
  ✅ **Use SVM (After SMOTE)** – Best trade-off between accuracy (98.4%) and fraud detection (78% recall).  
  ✅ **Logistic Regression (After SMOTE)** – If you prioritize recall (83%) over precision (73%).  

- **Avoid Naive Bayes** (too many false positives) and **Random Forest** (no improvement from SMOTE).

#### **Next Steps (If You Need Even Better Performance)**
1. **Hyperparameter Tuning**: Optimize SVM/LR for higher recall (e.g., adjust class weights).  
2. **Threshold Adjustment**: Lower decision threshold for LR to increase recall (at the cost of precision).  
3. **Feature Engineering**: Add more fraud-specific features (e.g., keyword flags like "work from home").  


##Hyperparameter(LR+SVM)

In [ ]:
svm_params = {
    'C': [0.1, 1, 10],               # Controls regularization (higher = less regularization)
    'class_weight': [None, 'balanced'],  # Adjusts for class imbalance
    'max_iter': [1000, 2000]          # Ensures convergence
}

In [ ]:
lr_params = {
    'C': [0.1, 1, 10],               # Inverse of regularization strength
    'class_weight': [None, 'balanced'],
    'solver': ['liblinear', 'saga']   # Solvers that handle L1/L2 regularization
}

In [ ]:
from imblearn.pipeline import Pipeline  # Critical fix!
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# 1. Define Pipelines with imblearn.Pipeline
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),  # TF-IDF first
    ('smote', SMOTE(random_state=42)),  # Then resample
    ('classifier', LinearSVC())
])

lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression())
])

# 2. Hyperparameter Grids
svm_params = {
    'classifier__C': [0.1, 1, 10],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__max_iter': [1000, 2000]
}

lr_params = {
    'classifier__C': [0.1, 1, 10],
    'classifier__class_weight': [None, 'balanced'],
    'classifier__solver': ['liblinear', 'saga']
}

# 3. GridSearchCV
svm_grid = GridSearchCV(
    svm_pipeline,
    param_grid=svm_params,
    cv=5,
    scoring='recall',
    n_jobs=-1
)

lr_grid = GridSearchCV(
    lr_pipeline,
    param_grid=lr_params,
    cv=5,
    scoring='recall',
    n_jobs=-1
)

# 4. Fit on raw text (TF-IDF happens inside the pipeline)
svm_grid.fit(X_train_text, y_train)
lr_grid.fit(X_train_text, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=5000,
                                                        stop_words='english')),
                                       ('smote', SMOTE(random_state=42)),
                                       ('classifier', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__class_weight': [None, 'balanced'],
                         'classifier__solver': ['liblinear', 'saga']},
             scoring='recall')

In [ ]:
# Best parameters
print("SVM Best Params:", svm_grid.best_params_)
print("LR Best Params:", lr_grid.best_params_)

# Predictions
svm_preds = svm_grid.predict(X_test_text)
lr_preds = lr_grid.predict(X_test_text)

# Classification reports
print("\nLogistic Regression (Tuned) Performance:")
print(classification_report(y_test, lr_preds))

print("\nSVM (Tuned) Performance:")
print(classification_report(y_test, svm_preds))

SVM Best Params: {'classifier__C': 0.1, 'classifier__class_weight': None, 'classifier__max_iter': 1000}
LR Best Params: {'classifier__C': 1, 'classifier__class_weight': None, 'classifier__solver': 'liblinear'}

Logistic Regression (Tuned) Performance:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      3395
           1       0.73      0.83      0.78       181

    accuracy                           0.98      3576
   macro avg       0.86      0.91      0.88      3576
weighted avg       0.98      0.98      0.98      3576


SVM (Tuned) Performance:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3395
           1       0.75      0.83      0.79       181

    accuracy                           0.98      3576
   macro avg       0.87      0.91      0.89      3576
weighted avg       0.98      0.98      0.98      3576



###Summary

Both models now achieve 83% recall (up from ~73-78% pre-tuning), meaning they catch 83% of fraudulent job postings.

SVM is slightly better than Logistic Regression:

Higher precision (75% vs. 73%) → Fewer false alarms.

Same recall (83%) → Equal fraud detection capability.

No need for class_weight='balanced': The best hyperparameters kept class_weight=None, suggesting SMOTE alone handled imbalance well.

#Streamlit App

In [ ]:
#Save both the model and vectorizer
import joblib
from imblearn.pipeline import Pipeline

# Save the ENTIRE pipeline (including TF-IDF + SMOTE + SVM)
joblib.dump(svm_grid.best_estimator_, "svm_tuned_smote_pipeline.joblib")

# (Optional: Save just the TF-IDF if needed separately)
joblib.dump(tfidf, "tfidf_vectorizer.joblib")

['tfidf_vectorizer.joblib']

In [ ]:
joblib.dump(lr_grid.best_estimator_, "lr_tuned_smote_pipeline.joblib")

['lr_tuned_smote_pipeline.joblib']

In [ ]:
# Load the full pipeline
model = joblib.load('svm_tuned_smote_pipeline.joblib')

# Input raw text
text = "Enter a job posting text here"

# Predict directly (NO vectorizer.transform)
prediction = model.predict([text])

print("Prediction:", "Fake" if prediction[0] == 1 else "Real")

Prediction: Real


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
#Skipped
%%writefile app.py
import streamlit as st
import joblib

# Load the entire pipeline (includes vectorizer + model)
model = joblib.load("svm_tuned_smote_pipeline.joblib")

st.title("Real/Fake Job Posting Detector")
st.write("Enter job description text below to predict whether it's real or fake.")

job_text = st.text_area("Job Description", "")

if st.button("Predict"):
    if job_text:
        # Just pass raw text directly into the pipeline
        result = model.predict([job_text])[0]
        label = "Fake" if result == 1 else "Real"
        st.success(f"The job posting is predicted to be: **{label}**")
    else:
        st.warning("Please enter job description text.")

Writing app.py


In [ ]:
#Model Selection
%%writefile app.py
import streamlit as st
import joblib

# Load both models
svm_model = joblib.load("svm_tuned_smote_pipeline.joblib")
lr_model = joblib.load("lr_tuned_smote_pipeline.joblib")  # Make sure you've saved this!

# App title and instructions
st.title("Real/Fake Job Posting Detector")
st.write("Enter a job description and choose a model to predict whether it's real or fake.")

# User input: job description
job_text = st.text_area("Job Description", "")

# Model selection dropdown
model_choice = st.selectbox("Select Model", ["SVM (Tuned)", "Logistic Regression (Tuned)"])

# Predict button
if st.button("Predict"):
    if job_text:
        # Choose model based on selection
        model = svm_model if model_choice == "SVM (Tuned)" else lr_model

        # Predict
        result = model.predict([job_text])[0]
        label = "Fake" if result == 1 else "Real"

        # Show result
        st.success(f"Using **{model_choice}**, the job posting is predicted to be: **{label}**")
    else:
        st.warning("Please enter job description text.")


Overwriting app.py


In [ ]:
# Install dependencies
!pip install streamlit pyngrok

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("2xdXYZyatVBx5nACW4Wt3qLaElg_6EgaYSTNihqfuR4ntxFvo")

# Open tunnel on port 8501 (Streamlit default)
public_url = ngrok.connect(8501)
print(f"Streamlit app public URL: {public_url}")

# Then run your Streamlit app as usual:
!streamlit run app.py

Streamlit app public URL: NgrokTunnel: "https://41a9-35-194-189-242.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.189.242:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1363, in main
    

#Gradio

In [ ]:
!pip install gradio

In [ ]:
#Load Trained Models
import joblib

# Load models
svm_model = joblib.load("svm_tuned_smote_pipeline.joblib")
lr_model = joblib.load("lr_tuned_smote_pipeline.joblib")

In [ ]:
#Define a Gradio Prediction Function
def predict_fake_job(text, model_name):
    if model_name == "SVM (Tuned)":
        model = svm_model
    else:
        model = lr_model

    pred = model.predict([text])[0]
    label = "Fake" if pred == 1 else "Real"
    return f"The job posting is predicted to be: {label}"


In [ ]:
#Build Gradio Interface

import gradio as gr

# Define inputs
textbox = gr.Textbox(lines=6, label="Job Description")
dropdown = gr.Dropdown(["SVM (Tuned)", "Logistic Regression (Tuned)"], label="Choose Model")

# Launch interface
demo = gr.Interface(fn=predict_fake_job,
                    inputs=[textbox, dropdown],
                    outputs="text",
                    title="Real/Fake Job Posting Detector",
                    description="Enter a job description and select a model to detect if it's real or fake.")

demo.launch(share=True)  # Use share=True for public URL in Colab


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1258e5d17ef8a1994d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
